In [ ]:
import yfinance as yf
import pandas as pd
import vectorbt as vbt
from ta.momentum import WilliamsRIndicator

tickers = (
    pd.read_csv("../../data/sp500_tickers.csv")["Symbol"].str.replace(".", "-").tolist()
)
data = yf.download(
    tickers,
    start="2023-01-01",
    end="2025-06-28",
    auto_adjust=True,
    threads=True,
    group_by="column",
)

price = data["Close"].dropna(how="all", axis=1)
high = data["High"][price.columns]
low = data["Low"][price.columns]
vol = data["Volume"][price.columns]

# rsi and macd
rsi_res = vbt.RSI.run(price)
rsi = rsi_res.rsi

macd_res = vbt.MACD.run(price)
macd_diff = macd_res.macd - macd_res.signal

# willians %R
williams_r = price.apply(
    lambda col: WilliamsRIndicator(
        high=high[col.name], low=low[col.name], close=col
    ).williams_r()
)

# avwap
LOOKBACK = 30

def compute_avwap_series(pr: pd.Series, vl: pd.Series) -> pd.Series:
    df = pd.DataFrame({"P": pr, "V": vl}).dropna()
    out = pd.Series(index=df.index, dtype="float64")
    for t in df.index:
        window = df.loc[:t].iloc[-LOOKBACK:]
        anchor = window["P"].idxmin()
        tpv = (df.loc[anchor:t, "P"] * df.loc[anchor:t, "V"]).cumsum()
        cum_v = df.loc[anchor:t, "V"].cumsum()
        out.loc[t] = tpv.iloc[-1] / cum_v.iloc[-1]
    return out


avwap = pd.DataFrame(
    {sym: compute_avwap_series(price[sym], vol[sym]) for sym in price.columns}
)
avwap_slope = avwap.diff().rolling(5).mean()

# entry/exit signals
entry = (
    (price > avwap)
    & (avwap_slope > 0)
    & (price < avwap * 1.05)
    & (macd_diff > 0)
    & (rsi < 70)
    & (williams_r > -80)
)

exit = ((avwap_slope <= 0) | (williams_r <= -50)).shift(1)

entry = entry.fillna(False).astype(bool)
exit  = exit.fillna(False).astype(bool)

pf = vbt.Portfolio.from_signals(
    close=price,
    entries=entry,
    exits=exit,
    init_cash=2_000,
    fees=0.001,
    slippage=0.001,
    freq="1D",
    cash_sharing=True
)

print("Total return:", pf.total_return())
print("Sharpe ratio:", pf.sharpe_ratio())
print("Max drawdown:", pf.max_drawdown())
pf.plot().show()


[*********************100%***********************]  503 of 503 completed


Total return: 0.7156959779778812
Sharpe ratio: 0.9656672382139225
Max drawdown: -0.30731553336630923


/var/folders/z7/l5mfp8zs6735wdd4hn4drvt80000gn/T/ipykernel_60229/3924458360.py:73: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/Users/robpickerill/scm/personal/github.com/robpickerill/moment2/.venv/lib/python3.13/site-packages/vectorbt/generic/plots_builder.py:339: UserWarning:

Subplot 'orders' does not support grouped data

/Users/robpickerill/scm/personal/github.com/robpickerill/moment2/.venv/lib/python3.13/site-packages/vectorbt/generic/plots_builder.py:339: UserWarning:

Subplot 'trade_pnl' does not support grouped data

